In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import matplotlib.pyplot as plt

## Решение задачи бинарной классификации

### Логистическая регрессия

y = {-1, 1}

$b(x) = \sigma(<w,x>)$, где $\sigma(z) = \frac{1}{1 + e^{-z}}$

Поработаем с уже известным нам датасетом Titanic. 

In [10]:
data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Почистим его и оставим только интересующие нас колонки. 

In [ ]:
# your code here

In [13]:
X = data.drop('Survived', axis=1)
y = data.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y)

Посмотрим, сбалансированная ли у нас выборка:

In [ ]:
y.value_counts()

Баланс не очень. Попробуем посмотреть сразу все скоры. 

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

In [ ]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

Как видим, мало отличается от работы с регрессией, синтаксис все тот же. Как sklearn понимает вообще, что у нас данные для регрессии или для классификации? А никак, мы можем спокойно применять регрессию к данным, предназначенным для классификации, и наоборот, решаем только мы сами, какой алгоритм куда больше подходит. 

### Нормализация данных

Давайте посмотрим один датасет (задача была классифицировать полосу автомобильной дороги - левую или правую - по координатам). 

In [ ]:
X = pd.read_csv('scaler_example.csv')
X.head()

Посмотрим, как выглядят точки из датасета.

In [ ]:
Xtrain = X[['lat','lon']].values
Ytrain = X['class'].values

plt.figure(figsize=(10,10))
plt.scatter(Xtrain[Ytrain==1][:,0],Xtrain[Ytrain==1][:,1],color='green')
plt.scatter(Xtrain[Ytrain==0][:,0],Xtrain[Ytrain==0][:,1],color='red')

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(Xtrain,Ytrain)

def plot_decision_line(Xtrain, Ytrain, clf, h):

    x_min, x_max = Xtrain[:,0].min()-0.01, Xtrain[:,0].max()+0.01
    y_min, y_max = Xtrain[:,1].min()-0.01, Xtrain[:,1].max()+0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10,10))
    plt.contourf(xx, yy, Z)

    Ytrain = np.array(Ytrain)
    plt.scatter(Xtrain[Ytrain==1][:,0],Xtrain[Ytrain==1][:,1],color='green')
    plt.scatter(Xtrain[Ytrain==0][:,0],Xtrain[Ytrain==0][:,1],color='red')

    a = clf.coef_[0][0]
    b = clf.coef_[0][1]
    c = clf.intercept_

    K = -a*1./b
    B = -c*1./b

    xx0 = np.linspace(x_min, x_max)
    yy0 = K * xx0 + B

    plt.scatter(xx0,yy0)
    plt.show()
    
plot_decision_line(Xtrain, Ytrain, clf, 0.001)

Видим, что линейный классификатор (SVM) абсолютно не справился с задачей. Попробуем масштабировать данные.

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
TrainScaled = sc.fit_transform(Xtrain) # сразу же обучается и применяет

clf.fit(TrainScaled, Ytrain)

SVC(kernel='linear')

In [ ]:
plot_decision_line(TrainScaled, Ytrain, clf, 0.01)

### Несбалансированные классы

Давайте порешаем финансовую задачку: будем предсказывать, возьмет клиент банка кредит или нет. 

In [17]:
data = pd.read_csv('bank-additional-full.csv', sep=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Для начала просто дропнем все нечисловые характеристики. 

In [40]:
X = data.select_dtypes(include=np.number)
y = data.y

Проверим распределение классов:

In [41]:
y.value_counts()

no     36548
yes     4640
Name: y, dtype: int64

Обучите обычную логистическую регрессию на этом датасете и выведите classification report.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y) # параметр stratify делает подвыборки с равномерным (по возможности) распределением классов

In [ ]:
# your code here

Какие выводы можете сделать на основании метрик?

А теперь давайте применим особую магию с class_weight. 

In [ ]:
# my code here